In [1]:
!pip install -q pypdf
!pip  install -q transformers einops accelerate langchain bitsandbytes sentence_transformers llama_index llama-index-llms-huggingface gradio pytube jiwer ReportLab weasyprint
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 872.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [3]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [4]:
# prompt: make a code to download a file and the wright it in a file

import requests
import os
if not os.path.exists('data'):
  os.makedirs('data')

def download_and_write_file(url, filename):
  response = requests.get(url)

  if response.status_code == 200:
    with open(filename, 'wb') as f:
      f.write(response.content)
  else:
    print('Failed to download file.')


download_and_write_file('https://arxiv.org/pdf/1706.03762.pdf','data/attention is all you need.pdf')

In [5]:
documents= SimpleDirectoryReader("/content/data").load_data()
# documents

In [6]:
system_prompt="""
You are a Q&A assistant. your goal is to answer questions as
accurately as possible bassed on the instruction and context provoided.
"""
# Default format supportableby LLama 2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [7]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

device = 'cuda' if torch.cuda.is_available() else "cpu"
llm= HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs= {"temperature":0,"do_sample":False},
    system_prompt= system_prompt,
    query_wrapper_prompt= query_wrapper_prompt,
    tokenizer_name= "google/gemma-1.1-2b-it",
    model_name= "google/gemma-1.1-2b-it",
    device_map= device,
    model_kwargs= {"torch_dtype": torch.float16, "quantization_config":BitsAndBytesConfig(
        load_in_4bit=True,  # Set to True for 4-bit quantization
        load_in_8bit=False,  # Set to True for 8-bit quantization
    )},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding

embed_model= LangchainEmbedding(
    HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
)

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
service_context= ServiceContext.from_defaults(
    chunk_size=1024,
    llm= llm,
    embed_model= embed_model
)

<ipython-input-9-e13bf89a3d6d>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context= ServiceContext.from_defaults(


In [10]:
import llama_index
from llama_index.core import VectorStoreIndex
from sentence_transformers import SentenceTransformer

In [11]:
embed_model = SentenceTransformer('all-mpnet-base-v2')

In [12]:
index= VectorStoreIndex.from_documents(documents,service_context= service_context)

In [13]:
query_engine= index.as_query_engine()

In [14]:
%%time
response= query_engine.query("what do you think about srk ?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


CPU times: user 6.46 s, sys: 871 ms, total: 7.33 s
Wall time: 11.9 s


In [15]:
print(response.response)

 The provided text does not contain any information regarding srk, so I am unable to provide an answer to this question.


In [ ]:
import whisper
import gradio as gr
from pytube import YouTube
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch
from weasyprint import HTML

model = whisper.load_model("base")
globvar=0

def echo(message, history):
  if "https://yout" in message or  "https://www.youtube.com/" in message:
    try:
      yt = YouTube(message)
      global globvar
      yt.streams.filter(only_video=True, file_extension='mp4')
      stream = yt.streams.get_by_itag(139)
      stream.download('',f"GoogleImagen{globvar}.mp4")
      result = model.transcribe(f"GoogleImagen{globvar}.mp4")
      # create_pdf_in_colab("my_pdf", result['text'])
      cont = result['text']
      filename = f"data/youtube{globvar}.pdf"
      globvar=globvar+1
      # Create a new PDF canvas
      c = canvas.Canvas(filename)

      # Add title to the PDF
      c.setFont("Helvetica", 24)
      c.drawCentredString(300, 750, 'title')
      # Define margins for content placement
      margin_left = 50
      margin_top = 700
      margin_right = 50

      # Check if using text frame or individual lines
      use_text_frame = len(cont.splitlines()) > 10  # Adjust threshold for long content

      if use_text_frame:
        # Use text frame for long content
        frame_width = 6.5 * inch  # Adjust width
        frame_height = 4.0 * inch  # Adjust height
        x_pos = margin_left
        y_pos = margin_top

        # Create text frame
        frame = c.beginFrame(x_pos, y_pos, frame_width, frame_height)

        # Set font and line spacing for frame
        c.setFont("Helvetica", 12)  # Adjust font size
        c.setLineSpacing(1.0)  # Adjust line spacing

        # Add content to the frame
        c.drawString(0, 0, cont)  # Assuming your content is in a variable

        # End the text frame
        c.endFrame()
      else:
        # Use individual lines for short content
        c.setFont("Helvetica", 12)  # Adjust font size
        # c.setLineSpacing(1.0)  # Adjust line spacing (multiplies font size for spacing)

        y_pos = margin_top

        for line in cont.splitlines():
          c.drawString(margin_left, y_pos, line)
          y_pos -= 18  # Adjust line spacing based on font size

      # Save and close the PDF
      c.save()
      return "Video Uploaded"
    except:
      return "Please provide a valid Youtube link...."
  else:
    documents= SimpleDirectoryReader("/content/data").load_data()
    index= VectorStoreIndex.from_documents(documents,service_context= service_context)
    query_engine= index.as_query_engine()
    response = query_engine.query(message)
    return response.response

demo = gr.ChatInterface(fn=echo, examples=["what is the meaning of attention is all you need ?", "https://www.youtube.com/watch?v=u47GtXwePms", "https://youtu.be/u47GtXwePms?si=roRjj6cKfednKOUS"], title="GDSC AI BOT")
demo.launch(debug=False)